In [1]:
# Scraping interactive images 
# https://towardsdatascience.com/image-scraping-with-python-a96feda8af2d
# This is for google chrome browser - help --> about google chrome my Version 85.0.4183.83

import os
import time
from time import sleep
from selenium import webdriver 
import io
from PIL import Image
import hashlib
import requests
from selenium.webdriver.chrome.options import Options


# Scrape images from google 
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)
        
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    DRIVER_PATH = '/usr/bin/chromedriver'

    wd = webdriver.Chrome('/usr/bin/chromedriver',options=chrome_options)
    
    

    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"
    
    #load the image
    wd.get(search_url.format(q = query))
    
    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)
        
        #get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector('img.Q4LuWd')
        number_results = len(thumbnail_results)
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            #try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
                
            except Exception:
                continue
                
            #extract image urls 
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))
                    
            image_count = len(image_urls)
            
            if len(image_urls) >= max_links_to_fetch:
                print(f"Found:{len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image linke, looking for more ...")
            time.sleep(30)
            return 
            
            loag_more_button = wd.find_elements_by_css_selector(".mye4qd")
            if loag_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")
                
        # move the result startpoint further down
        results_start = len(thumbnail_results)
        
    return image_urls

# Download the images 
def persist_image(folder_path:str, url:str):
    try:
        image_content = requests.get(url).content
    except Exception as e:
        print(f"Error - Could not download {url} - {e}")
    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, "wb") as f:
            image.save(f, "JPEG", quality = 85)
        print(f"SUCCESS - saved {url} as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

# Togehter
def search_and_download(search_term:str,driver_path:str, target_path='./images', number_images = 5):
    target_folder= os.path.join(target_path, '_'.join(search_term.lower().split(" ")))
    
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
        
    with webdriver.Chrome('/usr/bin/chromedriver',options=chrome_options) as wd:
        res = fetch_image_urls(search_term, number_images, wd = wd, sleep_between_interactions=2)
        
    for elem in res:
        persist_image(target_folder, elem)

In [5]:
# run the code:
DRIVER_PATH = '/usr/bin/chromedriver'
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

os.chdir('/home/ubuntu/gitscripts/FobiaPhilter/GoogleImage')

search_term = "clown"
search_and_download(search_term = search_term, driver_path = DRIVER_PATH, number_images = 100)

Found: 100 search results. Extracting links from 0:100
Found:100 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ_CN4IrN_Knfd0S5vEvtP8WvER97PXCvfSDw&usqp=CAU as ./images/clown/478b8c796a.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRUS9FWsd3rVwQJsVYGrXR1VLuyXCLyYDXkgQ&usqp=CAU as ./images/clown/042c6e1b26.jpg
SUCCESS - saved https://media.tegna-media.com/assets/WQAD/images/68699f10-6ae4-4b27-91cb-3a20736fe386/68699f10-6ae4-4b27-91cb-3a20736fe386_750x422.jpg as ./images/clown/22032b7ca4.jpg
SUCCESS - saved https://ychef.files.bbci.co.uk/976x549/p04chzvm.jpg as ./images/clown/b67317bb8f.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQo-AV-xXTpfVrI_P2mvDkmnKo4byWqygvVng&usqp=CAU as ./images/clown/7349a9b576.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQF9dECPlfKALdQ5JibMTNvJpaMUasoY-RkAQ&usqp=CAU as ./images/clown/eb561021b8.jpg
SUCCESS - saved https://enc

SUCCESS - saved https://www.adweek.com/wp-content/uploads/2019/05/bk-birthday-clowns-hed-page-2019-640x360.jpg as ./images/clown/2e45021ced.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTBDnPy8I47BId25F0D9n0WN9QbZPtXI_v4aw&usqp=CAU as ./images/clown/6e4b480e19.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTX-xQgX6zOOFRN4P3bk2GK2-RXflItdyRDzw&usqp=CAU as ./images/clown/89222ef547.jpg
SUCCESS - saved https://i.guim.co.uk/img/media/1933eadebc2259528d17419f5953d7d5de572ac5/0_68_2048_1229/master/2048.jpg?width=1200&height=1200&quality=85&auto=format&fit=crop&s=38291f67b4fbf53d6ea0fa2aabda08ab as ./images/clown/285680d9c9.jpg
SUCCESS - saved https://i.ytimg.com/vi/G2yvxmUrMRA/maxresdefault.jpg as ./images/clown/6b47790567.jpg
SUCCESS - saved https://dynaimage.cdn.cnn.com/cnn/c_fill,g_auto,w_1200,h_675,ar_16:9/https%3A%2F%2Fcdn.cnn.com%2Fcnnnext%2Fdam%2Fassets%2F141031122838-elam-creepy-clown-copycats-go-global-02-story-top.png as ./

In [3]:
# remove / paste every time


import os
import boto3

...
...
...

In [6]:
for root, dirs, files in os.walk('/home/ubuntu/gitscripts/FobiaPhilter/GoogleImage/images/clown'):
    for file in files:
        s3.Bucket("vidobject").upload_file(os.path.join(root, file), file)